In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
n_warehouses = 2
n_customers = 4

In [5]:
cost_matrix = np.array([[1,3,0.5,4],[2.5,5,1.5,2.5]])

In [7]:
cust_demands = np.array([35000, 22000, 18000, 30000])
warehouse_supply = np.array([60000, 80000])

In [10]:
# !pip install pulp


In [96]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,LpMinimize


In [115]:
model =LpProblem("Supply-Demand-Problem", LpMinimize)

In [116]:
variable_names = [str(i)+str(j) for j in range(1, n_customers+1) for i in range(1,n_warehouses+1)]
variable_names.sort()

In [117]:
variable_names

['11', '12', '13', '14', '21', '22', '23', '24']

In [118]:
# decsion varibale
Dec_var = LpVariable.matrix('X',indices=variable_names,lowBound=0,cat='Integer')

In [119]:
allocation = np.array(Dec_var).reshape(2,4)

In [120]:
allocation

array([[X_11, X_12, X_13, X_14],
       [X_21, X_22, X_23, X_24]], dtype=object)

In [121]:
# objective function
obj_func=lpSum(cost_matrix*allocation)
obj_func

1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0

In [122]:
model += obj_func

In [123]:
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

In [124]:
## constraints
# 1 warehouse
for i,v in enumerate(warehouse_supply):
    val=lpSum(allocation[i]) <=v
    print(val)
    model +=val,'warehouse_'+str(i)

X_11 + X_12 + X_13 + X_14 <= 60000
X_21 + X_22 + X_23 + X_24 <= 80000


In [125]:
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
SUBJECT TO
warehouse_0: X_11 + X_12 + X_13 + X_14 <= 60000

warehouse_1: X_21 + X_22 + X_23 + X_24 <= 80000

VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

In [126]:
## constraints
# 1 customer
for i,v in enumerate(cust_demands):
    val=lpSum(allocation[0][i]+allocation[1][i]) >=v
    print(val)
    model +=val,'customers'+str(i)

X_11 + X_21 >= 35000
X_12 + X_22 >= 22000
X_13 + X_23 >= 18000
X_14 + X_24 >= 30000


In [127]:
model

Supply-Demand-Problem:
MINIMIZE
1.0*X_11 + 3.0*X_12 + 0.5*X_13 + 4.0*X_14 + 2.5*X_21 + 5.0*X_22 + 1.5*X_23 + 2.5*X_24 + 0.0
SUBJECT TO
warehouse_0: X_11 + X_12 + X_13 + X_14 <= 60000

warehouse_1: X_21 + X_22 + X_23 + X_24 <= 80000

customers0: X_11 + X_21 >= 35000

customers1: X_12 + X_22 >= 22000

customers2: X_13 + X_23 >= 18000

customers3: X_14 + X_24 >= 30000

VARIABLES
0 <= X_11 Integer
0 <= X_12 Integer
0 <= X_13 Integer
0 <= X_14 Integer
0 <= X_21 Integer
0 <= X_22 Integer
0 <= X_23 Integer
0 <= X_24 Integer

In [128]:
# model.writeLP("Supply_demand_prob.lp")

In [129]:
model.solve()

1

In [130]:
status =  LpStatus[model.status]
print(status)

Optimal


In [131]:
print("Total Cost:", model.objective.value())
# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total Cost: 200000.0
X_11 = 35000.0
X_12 = 22000.0
X_13 = 3000.0
X_14 = 0.0
X_21 = 0.0
X_22 = 0.0
X_23 = 15000.0
X_24 = 30000.0


In [132]:
for i in range(n_warehouses):
    print("Warehouse ", str(i+1))
    print(lpSum(allocation[i][j].value() for j in range(n_customers)))

Warehouse  1
60000.0
Warehouse  2
45000.0


<h2>NExt PRoblem

In [3]:
from pulp import LpMaximize, LpProblem, LpStatus, lpSum, LpVariable,LpMinimize


In [4]:

df = pd.DataFrame([[60,20,0,70],
                     [80,10,40,95],
                     [140,0,100,230] 
                    ],index=['Swordman','Bowman','Horseman'],columns=['food','wood','gold','power'])

In [5]:
# cost chart of army
df

,food,wood,gold,power
Swordman,60,20,0,70
Bowman,80,10,40,95
Horseman,140,0,100,230


In [7]:
t_food=1200
t_wood=800
t_gold=600
resource=[t_food,t_wood,t_gold]

In [8]:
Swordman = LpVariable('Swordman',lowBound=0,cat='Integer')
Bowman = LpVariable('Bowman',lowBound=0,cat='Integer')
Horseman = LpVariable('Horseman',lowBound=0,cat='Integer')

In [6]:
model = LpProblem('Army',LpMaximize)

In [38]:
obj_func=lpSum(Swordman*70+Bowman*75+Horseman*230)
obj_func

75*Bowman + 230*Horseman + 70*Swordman + 0

In [39]:
model+=obj_func
model

Army:
MAXIMIZE
75*Bowman + 230*Horseman + 70*Swordman + 0
VARIABLES
0 <= Bowman Integer
0 <= Horseman Integer
0 <= Swordman Integer

In [41]:
model+=lpSum(Swordman*60+Bowman*80+Horseman*140)<=t_food
model+=lpSum(Swordman*20+Bowman*10+Horseman*0)<=t_wood
model+=lpSum(Swordman*0+Bowman*40+Horseman*100)<=t_gold

In [42]:
model

Army:
MAXIMIZE
75*Bowman + 230*Horseman + 70*Swordman + 0
SUBJECT TO
_C1: 80 Bowman + 140 Horseman + 60 Swordman <= 1200

_C2: 10 Bowman + 20 Swordman <= 800

_C3: 40 Bowman + 100 Horseman <= 600

VARIABLES
0 <= Bowman Integer
0 <= Horseman Integer
0 <= Swordman Integer

In [43]:
model.solve()

1

In [44]:
print("Total power", model.objective.value())
# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total power 1800.0
Bowman = 0.0
Horseman = 6.0
Swordman = 6.0


In [11]:
var = {'Swordman':Swordman,'Bowman':Bowman,'Horseman':Horseman}

In [16]:
model = LpProblem('Army',LpMaximize)

In [17]:

for i,v in enumerate(resource):
#     val = lpSum(df.iloc[:][i]<=v)
    l=df.iloc[:,i]
    val= lpSum(var[df.index[name]]*u for name,u in enumerate(l))<=v
    print(val)
    model+=val
    

80*Bowman + 140*Horseman + 60*Swordman <= 1200
10*Bowman + 20*Swordman <= 800
40*Bowman + 100*Horseman <= 600


In [18]:
obj_func=lpSum(Swordman*70+Bowman*75+Horseman*230)
obj_func

75*Bowman + 230*Horseman + 70*Swordman + 0

In [20]:
model+=obj_func

In [21]:
model

Army:
MAXIMIZE
75*Bowman + 230*Horseman + 70*Swordman + 0
SUBJECT TO
_C1: 80 Bowman + 140 Horseman + 60 Swordman <= 1200

_C2: 10 Bowman + 20 Swordman <= 800

_C3: 40 Bowman + 100 Horseman <= 600

VARIABLES
0 <= Bowman Integer
0 <= Horseman Integer
0 <= Swordman Integer

In [22]:
model.solve()

1

In [23]:
print("Total power", model.objective.value())
# Decision Variables

for v in model.variables():
    try:
        print(v.name,"=", v.value())
    except:
        print("error couldnt find value")

Total power 1800.0
Bowman = 0.0
Horseman = 6.0
Swordman = 6.0
